In [1]:
import os
import csv
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from utils import savefig

plt.rcParams['font.size'] = 14

count accuracy of each hyperparam setting

In [5]:
for j, seqlen in enumerate([4,8,12,16]):
    exp_dir = Path("./experiments/RL/Noise/Gamma09/figures/ValueMemoryGRU".format(seqlen))
    for noise in ["0", "02", "04", "06", "08", "1"]:
        for seqlen in [4, 8, 12, 16]:
            exp_name = "setup_seq{}_noise{}".format(seqlen, noise)
            accuracy = 0.0
            forward_asymmetry = 0.0
            temporal_factor = 0.0
            cnt = 0
            for i in range(20):
                if not os.path.exists(exp_dir / exp_name / str(i) / "contiguity_effect.csv"):
                    continue
                with open(exp_dir / exp_name / str(i) / "contiguity_effect.csv") as f:
                    reader = csv.reader(f)
                    for row in reader:
                        if float(row[0]) >= 0.5:
                            cnt += 1
                            accuracy += float(row[0])
                            forward_asymmetry += float(row[1])
                            temporal_factor += float(row[2])
            if cnt == 0:
                cnt = 1
            print(exp_name, cnt, np.round(accuracy / cnt, 4), np.round(forward_asymmetry / cnt, 4), np.round(temporal_factor / cnt, 4))
        print()
    print("#########################")


setup_seq4_noise0 20 0.9911 0.4748 0.3646
setup_seq8_noise0 20 0.9895 0.1168 0.6683
setup_seq12_noise0 20 0.9873 0.1047 0.7734
setup_seq16_noise0 20 0.9648 0.1787 0.7693

setup_seq4_noise02 20 0.9901 0.4062 0.3665
setup_seq8_noise02 20 0.9894 0.1187 0.6682
setup_seq12_noise02 20 0.985 0.1103 0.7682
setup_seq16_noise02 20 0.9411 0.2064 0.7368

setup_seq4_noise04 20 0.9902 0.3602 0.3956
setup_seq8_noise04 20 0.9892 0.1245 0.6655
setup_seq12_noise04 20 0.9822 0.1158 0.7659
setup_seq16_noise04 20 0.9261 0.2311 0.7118

setup_seq4_noise06 20 0.9793 0.8218 0.4589
setup_seq8_noise06 20 0.9893 0.1354 0.6586
setup_seq12_noise06 20 0.9303 0.2513 0.6635
setup_seq16_noise06 20 0.8701 0.3635 0.5698

setup_seq4_noise08 20 0.8497 0.9847 0.4965
setup_seq8_noise08 20 0.9725 0.5015 0.3989
setup_seq12_noise08 20 0.9239 0.4904 0.4118
setup_seq16_noise08 20 0.8756 0.4909 0.3962

setup_seq4_noise1 20 0.8612 0.9866 0.4966
setup_seq8_noise1 20 0.991 0.931 0.6892
setup_seq12_noise1 20 0.9401 0.5209 0.4151
setup

plot performance, FA, TF, proportion of index encoding of different seqlen and noise

In [6]:
def plot_bar(data, std, title, save_path):
    plt.figure(figsize=(6, 4), dpi=180)

    x = np.arange(0, 1.2, 0.2)
    plt.bar(x, data, yerr=std, width=0.1)
    plt.xticks(x, ["0", "0.2", "0.4", "0.6", "0.8", "1"])

    plt.xlabel("Noise proportion")
    plt.ylabel(title)

    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()

    savefig(save_path, title)

def plot_matrix(data, title, save_path, xticks=["0", "0.2", "0.4", "0.6", "0.8", "1"], yticks=["0", "0.3", "0.6", "0.9"],
                xlabel="Noise proportion", ylabel="Gamma"):
    assert len(xticks) == data.shape[1]
    assert len(yticks) == data.shape[0]

    plt.figure(figsize=(6, 4), dpi=180)

    plt.imshow(data, cmap='viridis', aspect='auto')
    plt.colorbar()
    plt.xticks(np.arange(len(xticks)), xticks)
    plt.yticks(np.arange(len(yticks)), yticks)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    plt.tight_layout()

    savefig(save_path, title)

In [7]:
seqlen = [4, 8, 12, 16]
noise = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [10]:
threshold = 0.7

# all models with performance > 70

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))
cnt = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        exp_path = Path("./experiments/RL/Noise/Gamma09/figures/ValueMemoryGRU/setup_seq{}_noise{}".format(s, str(n).replace(".", "")))
        for k in range(100):
            if not os.path.exists(exp_path / str(k) / "contiguity_effect.csv"):
                continue
            with open(exp_path / str(k) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        cnt[i, j] += 1
                        accuracy[i, j] += float(row[0])
                        forward_asymmetry[i, j] += float(row[1])
                        temporal_factor[i, j] += float(row[2])
                        data = np.load(exp_path / str(k) / "pc_selectivity_encoding.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold:
                            index_coding_prop[i, j] += 1.0
print(cnt)
cnt[cnt==0] = 1
accuracy /= cnt
forward_asymmetry /= cnt
temporal_factor /= cnt
index_coding_prop /= cnt
print(index_coding_prop)

xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/perf70")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)


[[20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 20.]
 [20. 20. 20. 20. 20. 18.]]
[[1.   1.   1.   1.   1.   1.  ]
 [0.05 0.05 0.   0.   0.   0.85]
 [0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.  ]]


In [25]:
threshold = 0.7

# top 20% performing models

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        acc, fa, tf, ic = [], [], [], []

        exp_path = Path("./experiments/RL/Noise/Seq{}/figures/ValueMemoryGRU/setup_{}noise{}_gamma{}".format(s, seq_setup_str[i], str(n).replace(".", ""), gamma[i][j]))
        for k in range(100):
            if not os.path.exists(exp_path / str(k) / "contiguity_effect.csv"):
                continue
            with open(exp_path / str(k) / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        acc.append(float(row[0]))
                        fa.append(float(row[1]))
                        tf.append(float(row[2]))
                        data = np.load(exp_path / str(k) / "pc_selectivity_encoding.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold:
                            ic.append(1.0)
                        else:
                            ic.append(0.0)
        if len(acc) == 0:
            continue
        acc = np.array(acc)
        fa = np.array(fa)
        tf = np.array(tf)
        ic = np.array(ic)
        top_acc_idx = np.argsort(acc)[-int(len(acc) * 0.2):]
        accuracy[i, j] = np.mean(acc[top_acc_idx])
        forward_asymmetry[i, j] = np.mean(fa[top_acc_idx])
        temporal_factor[i, j] = np.mean(tf[top_acc_idx])
        index_coding_prop[i, j] = np.mean(ic[top_acc_idx])


xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/toppf20")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)


same thing for n-back models

In [9]:
threshold = 0.7

# top 20% performing models

accuracy = np.zeros((4, 6))
forward_asymmetry = np.zeros((4, 6))
temporal_factor = np.zeros((4, 6))
index_coding_prop = np.zeros((4, 6))
cnt = np.zeros((4, 6))


for i, s in enumerate(seqlen):
    for j, n in enumerate(noise):
        acc, fa, tf, ic = [], [], [], []

        for k in range(50):
            exp_path = Path("./experiments/RL/NBack/VarySeq/figures/ValueMemoryGRU/setup_seq{}_noise{}_gamma{}-{}".format(s, str(n).replace(".", ""), gamma[i][j], k))
            # print(exp_path)
            if not os.path.exists(exp_path / "contiguity_effect.csv"):
                continue
            with open(exp_path / "contiguity_effect.csv") as f:
                reader = csv.reader(f)
                for row in reader:
                    if float(row[0]) >= 0.7:
                        cnt[i, j] += 1
                        acc.append(float(row[0]))
                        fa.append(float(row[1]))
                        tf.append(float(row[2]))
                        data = np.load(exp_path / "pc_selectivity_encoding.npz", allow_pickle=True)
                        if data['selectivity'][1][-1] >= threshold:
                            ic.append(1.0)
                        else:
                            ic.append(0.0)
        if len(acc) == 0:
            continue
        acc = np.array(acc)
        fa = np.array(fa)
        tf = np.array(tf)
        ic = np.array(ic)
        top_acc_idx = np.argsort(acc)
        accuracy[i, j] = np.mean(acc[top_acc_idx])
        forward_asymmetry[i, j] = np.mean(fa[top_acc_idx])
        temporal_factor[i, j] = np.mean(tf[top_acc_idx])
        index_coding_prop[i, j] = np.mean(ic[top_acc_idx])

print(cnt)

xticks = ["0", "0.2", "0.4", "0.6", "0.8", "1"]
yticks = [4, 8, 12, 16]
xlabel = "noise proportion"
ylabel = "sequence length"
figpath = Path("./figures/strategy/nback_all")

plot_matrix(accuracy, "accuracy", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(forward_asymmetry, "forward asymmetry", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(temporal_factor, "temporal factor", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
plot_matrix(index_coding_prop, "index coding proportion", figpath, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)


[[48. 50. 46. 49. 50. 50.]
 [42. 43. 38. 35. 38.  4.]
 [27. 30. 29. 22. 17. 24.]
 [31. 29. 25. 17. 24. 11.]]
